In [5]:
import h5py
import numpy as np
from tqdm import tqdm

In [8]:

# file_path = "/mnt/d/PhotonData/W-kedgescan/W-kedgescan.h5"

file_path = "W-kedgescan-background.h5"

with h5py.File(file_path, 'r') as f:
    raw_data_group = f['Raw Data']

    # 自动提取并排序所有数据集名（去掉前缀）
    dataset_keys = list(raw_data_group.keys())
    
    # 只保留以 '100kV-20mA-' 开头的
    dataset_names = [key.replace('100kV-20mA-', '') for key in dataset_keys if key.startswith('100kV-20mA-')]

    # 按 start energy 排序（例如 '20-24' → 20）
    def get_start_energy(name):
        return int(name.split('-')[0])
    
    dataset_names.sort(key=get_start_energy)



n_thresh_total = len(dataset_names) * 5  # 175
nrow, ncol = 24, 36
npix = nrow * ncol

print(dataset_names)

# 初始化左右晶体的堆栈：175帧 × 每帧864像素
left_stack = np.zeros((n_thresh_total, npix), dtype=np.float32)
right_stack = np.zeros((n_thresh_total, npix), dtype=np.float32)

frame_idx = 0


['20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', '100-104', '105-109', '110-114', '115-119', '120-124', '125-129', '130-134', '135-139', '140-145', '145-149']


In [9]:

with h5py.File(file_path, 'r') as f:
    raw_data_group = f['Raw Data']

    for name in tqdm(dataset_names, desc="Summing and Splitting"):
        full_key = '100kV-20mA-' + name
        data = raw_data_group[full_key][:]  # (10000, 5, 1728)

        # 时间维度上求和
        summed = np.sum(data, axis=0)  # (5, 1728)

        # 拆分左右两个晶体
        left = summed[:, :864]   # (5, 864)
        right = summed[:, 864:]  # (5, 864)

        # 写入堆栈
        for i in range(5):
            left_stack[frame_idx] = left[i]
            right_stack[frame_idx] = right[i]
            frame_idx += 1


Summing and Splitting: 100%|██████████| 26/26 [09:03<00:00, 20.90s/it]


In [10]:

# 重塑为 (帧数, 行, 列)
left_stack = left_stack.reshape(-1, nrow, ncol).astype(np.uint32)
right_stack = right_stack.reshape(-1, nrow, ncol).astype(np.uint32)

# 保存为 .raw 文件
left_stack.tofile("left_detector_summed.raw")
right_stack.tofile("right_detector_summed.raw")

print("写入完成：left_detector_summed.raw 和 right_detector_summed.raw")


写入完成：left_detector_summed.raw 和 right_detector_summed.raw
